In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import dataset as dd # custom dataset class
%matplotlib inline


In [ ]:
dataset, im_ref, im_us = dd.get_dataset(1)

In [ ]:
slice_to_show = 160

N, X, Y, C = im_us.shape

plt.imshow(im_ref[slice_to_show, :, :], cmap='gray')
plt.title('fully sampled image')
plt.axis('off')
plt.show()

coil_ims = np.transpose(im_us[slice_to_show, :, :, :], (2, 0, 1))
coil_ims_to_show = np.reshape(coil_ims[0:2, :, :], (2 * X, Y))

for ii in range(1, 4):
    coil_ims_to_show = np.concatenate((coil_ims_to_show, 
                                       np.reshape(coil_ims[2 * ii: 2 * (ii + 1), :, :], (2 * X, Y))), axis=1)

plt.imshow(coil_ims_to_show, cmap='gray')
plt.title('coil images')
plt.axis('off')
plt.show()

In [ ]:
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()


In [ ]:
# https://www.tensorflow.org/programmers_guide/using_gpu
# set CUDA_VISIBLE_DEVICES="" to force cpu
# https://stackoverflow.com/questions/46135499/how-to-properly-combine-tensorflows-dataset-api-and-keras
x = np.arange(4).reshape(-1, 1).astype('float32')
ds_x = tf.data.Dataset.from_tensor_slices(x).repeat().batch(4)
it_x = ds_x.make_one_shot_iterator()

y = np.arange(5, 9).reshape(-1, 1).astype('float32')
ds_y = tf.data.Dataset.from_tensor_slices(y).repeat().batch(4)
it_y = ds_y.make_one_shot_iterator()

input_vals = tf.keras.Input(tensor=it_x.get_next())
output = tf.keras.layers.Dense(1, activation='relu')(input_vals)
model = tf.keras.Model(inputs=input_vals, outputs=output)
model.compile('rmsprop', 'mse', target_tensors=[it_y.get_next()])
model.fit(steps_per_epoch=1, epochs=5, verbose=2)